Обучите простую рекуррентную нейронную сеть (без GRU/LSTM, без внимания) решать задачу дешифровки шифра Цезаря:
1. Написать алгоритм шифра Цезаря для генерации выборки (сдвиг на N каждой буквы). Например если N=2, то буква A переходит в букву C. Можно поиграться с языком на выбор (немецкий, русский и т.д.)
2. Создать архитектуру рекуррентной нейронной сети.
3. Обучить ее (вход - зашифрованная фраза, выход - дешифрованная фраза).
4. Проверить качество модели.	



In [2]:
import pandas as pd
import time
import torch

In [3]:
# Создаем алфавит
sym_eng =  'abcdefghijklmnopqrstuvwxyz '
N = 2

Алгоритм шифра Цезаря

In [9]:
def code_cesar(text:str, key:int):
    str_res = ''
    for row in text:
        for c in row:
            if c in sym_eng:
                ind_c = sym_eng.index(c)
                str_res+= sym_eng[(ind_c+key)%len(sym_eng)]
            else:
                str_res+=c
    return str_res
    
def de_code_cesar(text:str, key:int):
     str_res = ''
     for row in text:
        for c in row:
            if c in sym_eng:
                ind_c = sym_eng.index(c)
                str_res+= sym_eng[(ind_c-key)%len(sym_eng)]
            else:
                str_res+=c
     return str_res         


In [11]:
#тест
text_test=['maggie look whats that']
key = N
code_cesar(text_test, N)

'ociikgbnqqmbyjcvubvjcv'

In [13]:
sym_eng =  'abcdefghijklmnopqrstuvwxyz'

In [15]:
#тест
text_test='ociikg nqqm yjcvu vjcv'
de_code_cesar(text_test, N)

'maggie look whats that'

In [18]:
df = pd.read_csv('D:/файлы/data.csv')
df.head()

,Unnamed: 0,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count
0,0,10368,35,29,"Lisa Simpson: Maggie, look. What's that?",235000,True,9,5.0,Lisa Simpson,Simpson Home,"Maggie, look. What's that?",maggie look whats that,4.0
1,1,10369,35,30,Lisa Simpson: Lee-mur. Lee-mur.,237000,True,9,5.0,Lisa Simpson,Simpson Home,Lee-mur. Lee-mur.,lee-mur lee-mur,2.0
2,2,10370,35,31,Lisa Simpson: Zee-boo. Zee-boo.,239000,True,9,5.0,Lisa Simpson,Simpson Home,Zee-boo. Zee-boo.,zee-boo zee-boo,2.0
3,3,10372,35,33,Lisa Simpson: I'm trying to teach Maggie that ...,245000,True,9,5.0,Lisa Simpson,Simpson Home,I'm trying to teach Maggie that nature doesn't...,im trying to teach maggie that nature doesnt e...,24.0
4,4,10374,35,35,"Lisa Simpson: It's like an ox, only it has a h...",254000,True,9,5.0,Lisa Simpson,Simpson Home,"It's like an ox, only it has a hump and a dewl...",its like an ox only it has a hump and a dewlap...,18.0


In [30]:
#создаем корпус - phrases, состоящий из документов - text
phrases = df['normalized_text'].tolist()
phrases[:5]

['maggie look whats that',
 'lee-mur lee-mur',
 'zee-boo zee-boo',
 'im trying to teach maggie that nature doesnt end with the barnyard i want her to have all the advantages that i didnt have',
 'its like an ox only it has a hump and a dewlap hump and dew-lap hump and dew-lap']

In [33]:
#проходим по фразе, проверяем, что строка, разбиваем на буквы
#применяем функцию шифрования
text_code = [[code_cesar(c, N) for c in ph] for ph in phrases if type(ph) is str]

In [34]:
text_code[2]

['b', 'g', 'g', '-', 'd', 'q', 'q', ' ', 'b', 'g', 'g', '-', 'd', 'q', 'q']

In [41]:
#дешифрованная фраза, то что должны получить на выходе
text = [[c for c in ph] for ph in phrases if type(ph) is str]

In [43]:
text[2]

['z', 'e', 'e', '-', 'b', 'o', 'o', ' ', 'z', 'e', 'e', '-', 'b', 'o', 'o']

Создаем словарь, где ключи - это символы, из которых состояит алфавит, и значения уникальный индекс.
Добавляем None в набор ключей для непредвиденный ситуаций.

In [47]:
CHARS = set('abcdefghijklmnopqrstuvwxyz ')
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}

In [49]:
len(INDEX_TO_CHAR)

28

In [51]:
CHAR_TO_INDEX

{'none': 0,
 'v': 1,
 'r': 2,
 'i': 3,
 'n': 4,
 ' ': 5,
 'u': 6,
 'o': 7,
 'b': 8,
 'c': 9,
 's': 10,
 'k': 11,
 'z': 12,
 'a': 13,
 'm': 14,
 'd': 15,
 'y': 16,
 'e': 17,
 'j': 18,
 'h': 19,
 'x': 20,
 'g': 21,
 'p': 22,
 'l': 23,
 'q': 24,
 't': 25,
 'w': 26,
 'f': 27}

In [53]:
#задаем max длину, так как не можем работать с векторами разной длины - MAX_LEN
# создаем тензор с 0 размером нашей выборки и задаем макс длину
# проходим по документу и где длина менее MAX_LEN заполняем нулями
# i элемент текста (sample), j -элемент sample

In [55]:
MAX_LEN = 50
X = torch.zeros((len(text_code), MAX_LEN), dtype = int)

for i in range(len(text_code)):
	for j, w in enumerate(text_code[i]):
		if j >= MAX_LEN:
			break
		X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [56]:
X[:5]

tensor([[ 7,  9,  3,  3, 11, 21,  5,  4, 24, 24, 14,  5, 16, 18,  9,  1,  6,  5,
          1, 18,  9,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 4, 21, 21,  0,  7, 26, 25,  5,  4, 21, 21,  0,  7, 26, 25,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 8, 21, 21,  0, 15, 24, 24,  5,  8, 21, 21,  0, 15, 24, 24,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [11,  7,  5,  1, 25, 13, 11, 22,  3,  5,  1, 24,  5,  1, 21,  9, 17, 18,
          5,  7,  9,  3,  3, 11, 21,  5,  1, 18,  9,  1,  5, 22,  9,  1, 26, 25,
         21,  5, 27, 24, 21,  6, 22,  1,  5, 21, 22, 27,  5, 16],
        [11,  1,  6,  5,  4, 11, 14, 21,  5,  9, 22,  5, 24, 12,  5, 24, 22,  4,
       

In [61]:
X.shape

torch.Size([10891, 50])

In [63]:
MAX_LEN = 50
Y = torch.zeros((len(text), MAX_LEN), dtype = int)

for i in range(len(text)):
	for j, w in enumerate(text[i]):
		if j >= MAX_LEN:
			break
		Y[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [64]:
Y.shape

torch.Size([10891, 50])

In [67]:
Y[:5]

tensor([[14, 13, 21, 21,  3, 17,  5, 23,  7,  7, 11,  5, 26, 19, 13, 25, 10,  5,
         25, 19, 13, 25,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [23, 17, 17,  0, 14,  6,  2,  5, 23, 17, 17,  0, 14,  6,  2,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [12, 17, 17,  0,  8,  7,  7,  5, 12, 17, 17,  0,  8,  7,  7,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 3, 14,  5, 25,  2, 16,  3,  4, 21,  5, 25,  7,  5, 25, 17, 13,  9, 19,
          5, 14, 13, 21, 21,  3, 17,  5, 25, 19, 13, 25,  5,  4, 13, 25,  6,  2,
         17,  5, 15,  7, 17, 10,  4, 25,  5, 17,  4, 15,  5, 26],
        [ 3, 25, 10,  5, 23,  3, 11, 17,  5, 13,  4,  5,  7, 20,  5,  7,  4, 23,
       

In [72]:
# делим на train/test
num_trn = int(0.75*len(X))                
X_trn, Y_trn = X[:num_trn], Y[:num_trn]
X_test, Y_test = X[num_trn:], Y[num_trn:]

Модель нейронной сети:
1.Embedding 30
2.Скрытая ячейка - 128
3. Полносвязный слой из состояния rnn в букву

In [75]:
class RNNModel(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        self.embed = torch.nn.Embedding(len(INDEX_TO_CHAR) + N, 32)
        self.rnn = torch.nn.RNN(32, 128, batch_first=True)
        self.linear = torch.nn.Linear(128, len(INDEX_TO_CHAR) + N)

    def forward(self, sentence, state=None):
        embed = self.embed(sentence)
        o, h = self.rnn(embed)
        return self.linear(o)

In [55]:
model = RNNModel()
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

Обучение модели на 20 эпохах

In [57]:
for epoch in range(20):
    train_loss, train_acc, iter_num = .0, .0, .0
    start_epoch_time = time.time()
    model.train()
    for i in range(int(len(X_trn) / 100)):
        batch = X_trn[i * 100:(i + 1) * 100]
        X_trn_batch = batch
        Y_trn_batch = Y[i * 100:(i + 1) * 100]
        Y_trn_batch = Y_trn_batch.view(1, -1).squeeze()
        optimizer.zero_grad() 
        out = model.forward(X_trn_batch) 
        #print(out.shape)
        out = out.view(-1, len(INDEX_TO_CHAR) + N)
        #print(out.shape)
        #print(Y_batch.shape)
        l = loss(out, Y_trn_batch)
        train_loss += l.item()
        batch_acc = (out.argmax(dim=1) == Y_trn_batch)
        train_acc += batch_acc.sum().item() / batch_acc.shape[0]
        l.backward()
        optimizer.step()
        iter_num += 1
    print(
        f"Epoch: {epoch}, loss: {train_loss:.4f}, acc: "
        f"{train_acc / iter_num:.4f}",
        end=" | "
    )



Epoch: 0, loss: 25.3414, acc: 0.9556 | Epoch: 1, loss: 1.9452, acc: 0.9980 | Epoch: 2, loss: 1.0299, acc: 0.9986 | Epoch: 3, loss: 0.6893, acc: 0.9995 | Epoch: 4, loss: 0.5054, acc: 0.9996 | Epoch: 5, loss: 0.3919, acc: 0.9996 | Epoch: 6, loss: 0.3160, acc: 0.9996 | Epoch: 7, loss: 0.2620, acc: 0.9997 | Epoch: 8, loss: 0.2217, acc: 0.9999 | Epoch: 9, loss: 0.1907, acc: 1.0000 | Epoch: 10, loss: 0.1665, acc: 1.0000 | Epoch: 11, loss: 0.1472, acc: 1.0000 | Epoch: 12, loss: 0.1316, acc: 1.0000 | Epoch: 13, loss: 0.1188, acc: 1.0000 | Epoch: 14, loss: 0.1081, acc: 1.0000 | Epoch: 15, loss: 0.0992, acc: 1.0000 | Epoch: 16, loss: 0.0915, acc: 1.0000 | Epoch: 17, loss: 0.0849, acc: 1.0000 | Epoch: 18, loss: 0.0792, acc: 1.0000 | Epoch: 19, loss: 0.0742, acc: 1.0000 | 

Проверяем на тестовых данных

In [61]:
for epoch in range(20):
    start_epoch_time = time.time()
    test_loss, test_acc, iter_num = .0, .0, .0
    model.eval()
    
    Y_test = Y_test.view(1, -1).squeeze()
    out = model.forward(X_test)
    #print(out.shape)
    out = out.view(-1, len(INDEX_TO_CHAR) + N)
    #print(out.shape)
    #print(Y_batch.shape)
    l = loss(out, Y_test)
    test_loss += l.item()
    batch_acc = (out.argmax(dim=1) == Y_test)
    test_acc += batch_acc.sum().item() / batch_acc.shape[0]
    l.backward()
    optimizer.step()
    iter_num += 1
    print(
        f"Epoch: {epoch}, loss: {test_loss:.4f}, acc: "
        f"{test_acc / iter_num:.4f}",
        end=" | "
    )

Epoch: 0, loss: 2.4590, acc: 0.2378 | Epoch: 1, loss: 1.8859, acc: 0.5599 | Epoch: 2, loss: 1.0376, acc: 0.7246 | Epoch: 3, loss: 0.6985, acc: 0.8544 | Epoch: 4, loss: 0.3765, acc: 0.9483 | Epoch: 5, loss: 0.2180, acc: 0.9668 | Epoch: 6, loss: 0.1195, acc: 0.9849 | Epoch: 7, loss: 0.0677, acc: 0.9905 | Epoch: 8, loss: 0.0422, acc: 0.9956 | Epoch: 9, loss: 0.0301, acc: 0.9954 | Epoch: 10, loss: 0.0252, acc: 0.9948 | Epoch: 11, loss: 0.0223, acc: 0.9947 | Epoch: 12, loss: 0.0179, acc: 0.9960 | Epoch: 13, loss: 0.0141, acc: 0.9970 | Epoch: 14, loss: 0.0127, acc: 0.9974 | Epoch: 15, loss: 0.0126, acc: 0.9973 | Epoch: 16, loss: 0.0138, acc: 0.9966 | Epoch: 17, loss: 0.0153, acc: 0.9963 | Epoch: 18, loss: 0.0158, acc: 0.9966 | Epoch: 19, loss: 0.0151, acc: 0.9973 | 

In [ ]:
loss уменьшается с каждой эпохой и на train и на test, accuracy растет.

In [65]:
print(f"Test accuracy is : {test_acc:.4f}")

Test accuracy is : 0.9973


In [ ]:
accuracy ~ 1, loss стермится к 0 - качество модели удовлетворительное